In [ ]:
import requests

def get_weather(city: str):
    api_key = '118b63f0faa9cb1504b34a6b6810b30b'  # Replace with your OpenWeatherMap API key
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200:
        temperature = data['main']['temp']
        description = data['weather'][0]['description']
        return f'The temperature in {city} is {temperature}°C with {description}.'
    else:
        return f'Failed to get data for {city}. Error: {data.get("message", "Unknown error")}'


# File: ~/airflow/dags/weather_dag.py
from airflow import DAG
from airflow.operators.python import PythonOperator  # Updated import for PythonOperator
from datetime import datetime


# Define default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 1, 1),
    'retries': 1,
}

# Define the DAG
dag = DAG(
    'weather_dag',
    default_args=default_args,
    description='A simple weather-fetching DAG',
    schedule_interval='@daily',  # Change to your desired schedule
    catchup=False,
)

# Define the PythonOperator that runs your get_weather function
weather_task = PythonOperator(
    task_id='get_weather_task',
    python_callable=get_weather,
    op_args=['London'],  # Pass the city name here
    dag=dag,
)

weather_task
